In [10]:
import random
import math

class crypto:
    P = 2 ** 256 - 2 ** 32 - 2 ** 9 - 2 ** 8 - 2 ** 7 - 2 ** 6 - 2 ** 4 - 1  # prime number for modulus operations
    order = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141  # order for the elliptic curve y^2=x^3+7 ,used in bitcoin
    Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240  # x co-ordinate of generating point of secp256k1 i.e. curve used in bitcoin
    Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424  # y co-ordinate of generating point of secp256k1 i.e. curve used in bitcoin

    # mod inverse
    # Extended Euclidean Algorithm for finding mod inverse
    def modinv(self,a, n):
        lm, hm = 1, 0
        low, high = a % n, n
        while low > 1:
            ratio = high // low
            nm = hm - lm * ratio
            new = high - low * ratio
            hm = lm
            high = low
            lm = nm
            low = new
        return lm % n

    # Elliptic curve point addition
    def ECadd(self,xp, yp, xq, yq):
        # point addition will not work if both the point are same. So, point doubling is required
        if xp == xq and yp == yq:
            return self.ECdouble(xp, yp)

        m = ((yq - yp) * self.modinv(xq - xp, self.P)) % self.P
        xr = (m * m - xp - xq) % self.P
        yr = (m * (xp - xr) - yp) % self.P
        return (xr, yr)

    # Elliptic curve point doubling
    def ECdouble(self,xp, yp):
        l = ((3 * xp * xp) * self.modinv(2 * yp, self.P)) % self.P
        X = (l * l - 2 * xp) % self.P
        Y = (l * (xp - X) - yp) % self.P
        return X, Y

    # Elliptic curve point multiplication
    ##1. scaler multiplication with generator point
    def ECmult(self,scaler):
        if scaler == 0:
            return 0, 0
        _2pX = [0] * 258
        _2pY = [0] * 258
        _2pX[0], _2pY[0] = self.Gx, self.Gy
        _X = self.Gx
        _Y = self.Gy
        for i in range(1, 257):
            _2pX[i], _2pY[i] = self.ECdouble(_2pX[i - 1], _2pY[i - 1])

        index = 0
        while not (scaler & 1):
            index += 1
            scaler >>= 1
        _X = _2pX[index]
        _Y = _2pY[index]
        scaler >>= 1
        index += 1
        while scaler > 0:
            if scaler & 1:
                _X, _Y = self.ECadd(_X, _Y, _2pX[index], _2pY[index])
            scaler >>= 1
            index += 1
        return _X, _Y

    # fast EC scaler multiplication 4x faster than previous method
    # this can be improved more by catching precalculating group by few bits.
    def ECmult_fast(scaler):
        if scaler==0:
            return 0,0
        if(scaler==1):
            return Gx
        scaler-=1
        _2px=Gx
        _2py=Gy
        _x=Gx
        _y=Gy
        while(scaler):
            if scaler&1:
                _x,_y=ECadd(_x,_y,_2px,_2py)
            scaler>>=1
            _2px,_2py=ECdouble(_2px,_2py)
        return _x,_y

    ##2. scaler multiplication with other point on secp256k1 curve
    ###Example 2*(4G)=8G 4*(5G)=20G etc.
    def ECmultp(self,Sx, Sy, scaler):
        _2pX = [0] * 258
        _2pY = [0] * 258
        _2pX[0], _2pY[0] = Sx, Sy
        _X = Sx
        _Y = Sy
        for i in range(1, 257):
            _2pX[i], _2pY[i] = self.ECdouble(_2pX[i - 1], _2pY[i - 1])

        index = 0
        while not (scaler & 1):
            index += 1
            scaler >>= 1
        _X = _2pX[index]
        _Y = _2pY[index]
        scaler >>= 1
        index += 1

        while scaler > 0:
            if scaler & 1:
                _X, _Y = self.ECadd(_X, _Y, _2pX[index], _2pY[index])
            scaler >>= 1
            index += 1
        return _X, _Y

    # ECscalerDivide divide a ECpoint by a scaler number
    # e.g. 8G/2=4G
    def ECscalerDivide(self,x,y,scaler):
      return self.ECmultp(x,y,self.modinv(scaler,self.order))

    def ECxToyNotWorking(self,x):
      a = (x**3 + 7)%self.P
      ##_1_2 = self.modinv(2,self.order)
      y = self.tonelli_shanks(a,self.P)#pow(a,_1_2,self.P)%self.P
      if(y==None):
        return 0,0
      return y,(self.P-y)%self.P

    # find possible y coordinates w.r.t x coordinate
    # y^2 = x^3+7
    # y = pow_mod(x^3+7,P+1//4)
    def ECxToy(self,x):
      a = (pow(x, 3, self.P) + 7) % self.P
      if(self.tonelli_shanks(a,self.P)==None):
        print("sqrt doesnot exists")
      y = pow(a, (self.P+1)//4, self.P)
      return y,(self.P-y)%self.P


    #Generate a private key
    def generate_private_key(self):
        st = 1 << 254
        en = self.order - 1
        rn = random.randrange(st, en)
        (rx, ry) = self.ECmult(rn)
        rn=(rx+ry)%self.order
        (rx, ry) = self.ECmult(rn)
        return ry;

    #private key to public key conversion i.e. x coordinate of G*priv
    def private_to_public(self,priv_k):
        return self.ECmult(priv_k)

    ## Signing message and returning random part and signature
    def sign_message(self,msg,rn,prv_k):
        print("signing....")
        #order=115792089237316195423570985008687907852837564279074904382605163141518161494337
        st=1<<254
        en=self.order-1
        #rn=random.randrange(st, en)
        #print(rn)
        (rx,ry)=self.ECmult(rn)
        #print(rx,ry)
        sign=(((rx*prv_k)%self.order+msg)*self.modinv(rn,self.order))%self.order
        print("rx:",rx)
        print("sign:",sign)
        return (rx,sign)

    #verifying message
    def verify_message(self,s,r,msg,pub_x,pub_y):
        print("verifying...")
        m_s=(msg*self.modinv(s,self.order))%self.order
        print("m_s:",m_s)
        r_s=(r*self.modinv(s,self.order))%self.order
        print("r_s:",r_s)
        (x,y)=self.ECmult(m_s)
        (x2,y2)=self.ECmultp(pub_x,pub_y,r_s)
        (x3,y3)=self.ECadd(x,y,x2,y2)
        print("(x,y):",(x,y))
        print("(x2,y2):",(x2,y2))
        print("(x3,y3):",(x3,y3))
        if(x3==r):
            #print("valid")
            return True
        else:
            #print("invalid")
            return False

    #encryption of message to be send
    #Currently this is only xor(NOT SECURE), it is to be changed to ECadd & ECsub
    def encrypt_message(self,msg,r,prv_k,pub_k):
        (px,py)=self.ECmultp(pub_k[0],pub_k[1],prv_k)
        (rx,ry)=self.ECmultp(px,py,r)
        enc_msg=0
        l=0
        cnt=0
        for ch in msg:
            cnt+=1
            # print(ord(ch))
            l<<=8
            l+=ord(ch)
            if(cnt==30):
                cnt=0
                l=l^rx
                enc_msg <<= 256
                enc_msg|=l
                l=0
        if(l):
            l = l ^ rx
            enc_msg <<= 256
            enc_msg |= l

        # print(l)
        # enc_msg=l+rx
        # print(enc_msg)
        return enc_msg
        # encoded_string = msg.encode()
        # byte_array = bytearray(encoded_string)
        # print(byte_array)


    # decryption of the received message
    # Currently this is only xor(NOT SECURE), it is to be changed to ECadd & ECsub Concept
    def decrypt_message(self,enc_msg,r,prv_k,pub_k):
        (px, py) = self.ECmultp(pub_k[0], pub_k[1], prv_k)
        (rx, ry) = self.ECmultp(px, py, r)
        msg=""
        while(enc_msg):
            l=enc_msg&((1<<256)-1)^rx
            while(l):
                ch_i=l&255
                msg=chr(ch_i)+msg
                l>>=8
            enc_msg>>=256
        #print(msg)
        return msg

    # Elliptic curve point subtraction
    def ECsubtract(self,p1x,p1y,p2x,p2y):
      if(p1x==p2x):
        print("invalid point subtraction")
        return (0,0)
      return self.ECadd(p1x,p1y,p2x,(self.P-p2y)%self.P)

    def isValidPoint(self,x,y):
      x3_7 = ((x**3)+7)%self.P
      y2 = (y**2)%self.P
      return x3_7==y2

    def legendre_symbol(self,a, p):
      ls = pow(a, (p - 1) // 2, p)
      return -1 if ls == p - 1 else ls

    def tonelli_shanks(self,n, p):
      if self.legendre_symbol(n, p) != 1:
          return None  # No square root exists

      q = p - 1
      s = 0
      while q % 2 == 0:
          q //= 2
          s += 1

      if s == 1:
          return pow(n, (p + 1) // 4, p)

      for z in range(2, p):
          if self.legendre_symbol(z, p) == -1:
              break

      c = pow(z, q, p)
      r = pow(n, (q + 1) // 2, p)
      t = pow(n, q, p)
      m = s

      while True:
          if t == 0:
              return 0
          if t == 1:
              return r
          i = 0
          zz = t
          while zz != 1 and i < m:
              zz = (zz * zz) % p
              i += 1
          b = pow(c, 1 << (m - i - 1), p)
          r = (r * b) % p
          t = (t * b * b) % p
          c = (b * b) % p
          m = i


    # encryption of message using point addition and substraction
    # Encrypted format: {point,encrypted_data}
    # algo:
    #   Generate random integer, k
    #   Multiply with G, P1 = K.G
    #   Encrypting message
    #        read 255bits or less from the message Mx
    #        generate encrypted msg i.e. Emx = Mx+k.receiver_pubk, append 0 if y is odd and 1 otherwise
    #        keep Appending 257bits Emx in the encrypted message

    # Encrypting every 31bytes generates 33bytes encrypted data, which is ~6% increase in size.
    # Can be done with 255bits to 257bits which is ~0.7% increase in size.

    def encrypt2_0(self,msg,receiver_pubk,isUsingFirst):
      print("\n################ EnCrypt ###################")
      bytes_value = msg.encode('utf-8')
      ln=len(bytes_value)
      n_blocks=math.ceil(ln/31.0)
      print(ln,n_blocks,bytes_value)

      enc_k = self.generate_private_key()#a random number k
      enc_p1 = self.private_to_public(enc_k)# k.G

      total_encrypted_msg = b''

      for i in range(n_blocks):
        msg=bytes_value[i*31:i*31+31]
        msgint=int.from_bytes(msg, byteorder='big')
        print(i," block:",msg)
        print("msgint",msgint)

        msgys = self.ECxToy(msgint)
        msgys2 = self.ECxToyNotWorking(msgint)
        # choose y as the odd one
        msgy_choosen = msgys[0]#default
        if(isUsingFirst != 1):
           msgy_choosen = msgys[1]

        # msgy_odd = msgys[0]
        # if((msgy_odd & 1)!= 1):
        #   msgy_odd = msgys[1]
        #print("msgys2",msgys2)
        print("msgys:",msgys," choosen:",msgy_choosen)
        print("validity:",self.isValidPoint(msgint,msgys[0]))
        print("validity:",self.isValidPoint(msgint,msgys[1]))
        print("validity 2:",self.isValidPoint(msgint,msgys2[0]))
        print("validity 2:",self.isValidPoint(msgint,msgys2[1]))

        # creating k.Pb
        kPb = self.ECmultp(receiver_pubk[0],receiver_pubk[1],enc_k)
        print("k.Pb:",kPb)

        # add kpb with msg
        msg_kPb = self.ECadd(msgint,msgy_choosen,kPb[0],kPb[1])
        print("msg_kPb:",msg_kPb)

        enc_msg_chunk  = msg_kPb[0].to_bytes(32, byteorder='big')
        print("enc_msg_chunk:",enc_msg_chunk)

        if (msg_kPb[1]&1 == 1):
          enc_msg_chunk = b'1'+enc_msg_chunk
        else:
          enc_msg_chunk = b'0'+enc_msg_chunk

        print(len(enc_msg_chunk),"enc_msg_chunk:",enc_msg_chunk)
        total_encrypted_msg = total_encrypted_msg + enc_msg_chunk

      return (enc_p1,total_encrypted_msg)

    def decrypt2_0(self,enc_points,receiver_privk):
      print("\n################ DeCrypt ###################")
      enc_p1 = enc_points[0]
      enc_msg = enc_points[1]
      print("enc_p1:",enc_p1)
      print("enc_msg:",enc_msg)

      # k.G.priv
      kGprvk = self.ECmultp(enc_p1[0],enc_p1[1],receiver_privk)
      print("kGprvk:",kGprvk)

      total_decrypted_str = ""

      #iterate through all blocks
      ln=len(enc_msg)
      n_blocks=math.ceil(ln/33.0)
      for i in range(n_blocks):
        enc_msg_block=enc_msg[i*33:i*33+33]
        y_sign=enc_msg_block[0:1]
        enc_msg_block=enc_msg_block[1:]
        print("y_sign",y_sign,"enc_msg_block:",enc_msg_block)

        y_sign_int=int.from_bytes(y_sign, byteorder='big')
        enc_msg_block_int=int.from_bytes(enc_msg_block, byteorder='big')
        print("y_sign_int",y_sign_int,"enc_msg_block_int:",enc_msg_block_int)

        # find y for enc_msg_block_int to form the point
        ys_for_enc_msg_block_int = self.ECxToy(enc_msg_block_int)
        print("ys_for_enc_msg_block_int:",ys_for_enc_msg_block_int)
        y_for_enc_msg_block_int=ys_for_enc_msg_block_int[0]

        if(((y_for_enc_msg_block_int&1)^(y_sign_int&1))!=0):
          y_for_enc_msg_block_int=ys_for_enc_msg_block_int[1]
        print("y_for_enc_msg_block_int:",y_for_enc_msg_block_int)

        # decrypt the message
        dec_msg_int = self.ECsubtract(enc_msg_block_int,y_for_enc_msg_block_int,kGprvk[0],kGprvk[1])
        print(dec_msg_int)
        msg_bytes_value = dec_msg_int[0].to_bytes(32, byteorder='big')
        print("msg_bytes_value:",msg_bytes_value)

        null_byte_str = b'\x00'.decode('utf-8')
        string_value = msg_bytes_value.decode('utf-8', errors='replace')
        string_value = string_value.lstrip(null_byte_str)
        print("string_value:",string_value)
        total_decrypted_str = total_decrypted_str + string_value
      return total_decrypted_str

In [7]:
cr = crypto()
for i in range(1,25):
  print(i,":",cr.tonelli_shanks(i,23))

1 : 1
2 : 18
3 : 16
4 : 2
5 : None
6 : 12
7 : None
8 : 13
9 : 3
10 : None
11 : None
12 : 9
13 : 6
14 : None
15 : None
16 : 4
17 : None
18 : 8
19 : None
20 : None
21 : None
22 : None
23 : None
24 : 1


In [11]:
# 15 April 2024
# testing encryption/decryption 2.0

cr = crypto()
pk = cr.generate_private_key()
pubk = cr.private_to_public(pk)
print(pubk)
print(pubk[0])

for i in range(10):
  print("###",i,"\nUsing First point")
  encrypted_points = cr.encrypt2_0("hello world hello"+str(i),pubk,1)
  print("encrypted:",encrypted_points)
  print("\n\n########################in transit ####################\n",encrypted_points)
  decrypted_msg = cr.decrypt2_0(encrypted_points,pk)
  print("decrypted:",decrypted_msg)

  # print("###\nUsing second point")
  # encrypted_points = cr.encrypt2_0("hello world hello"+str(i),pubk,0)
  # print("encrypted:",encrypted_points)
  # print("\n\n########################in transit ####################\n",encrypted_points)
  # decrypted_msg = cr.decrypt2_0(encrypted_points,pk)
  # print("decrypted:",decrypted_msg)

(39384949731739144422689952512240699874968308527111888425326183178293557148301, 66487413866715142992358501770617662630142761207195850698539498262825807273063)
39384949731739144422689952512240699874968308527111888425326183178293557148301
### 0 
Using First point

################ EnCrypt ###################
18 1 b'hello world hello0'
0  block: b'hello world hello0'
msgint 9094190375607605659387419711506762278137648
sqrt doesnot exists
msgys: (104679496316163136672995777642902166819261831922204702211946131494506576209166, 11112592921153058750575207365785741034008152743435861827511452513402258462497)  choosen: 104679496316163136672995777642902166819261831922204702211946131494506576209166
validity: False
validity: False
validity 2: False
validity 2: False
k.Pb: (115523680331942606768413559114119659206838620993833606328679391822458343729225, 56168462070927612462847264384083419504086496234426051050739051880978414153985)
msg_kPb: (91725621635690744256302565930882255783899236533491808581728306

In [ ]:
# 15 April 2024
# testing substraction

cr = crypto()


# print("pubk:",pubk)
# genY = cr.ECxToy(pubk[0])
# print("genY:",genY)
# n_y = (cr.P-pubk[1])%cr.P
# print("n_y:",n_y)
# point = cr.ECadd(pubk[0],pubk[1],pubk[0],n_y)
# print("point:",point)
cnt=0
neg_cnt=0

for i in range(100):
  cnt+=1
  pk = cr.generate_private_key()
  p1 = cr.private_to_public(pk)

  pk2 = cr.generate_private_key()
  p2 = cr.private_to_public(pk2)

  p3 = cr.ECadd(p1[0],p1[1],p2[0],p2[1])

  p4 = cr.ECsubtract(p3[0],p3[1],p2[0],p2[1])
  # print("p1:",p1)
  # print("p2:",p2)
  # print("p3:",p3)
  # print("p4:",p4)
  #print(p1==p4)
  if(p1!=p4):
    neg_cnt+=1

print("cnt:",cnt," neg_cnt:",neg_cnt)


#Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240  # x co-ordinate of generating point of secp256k1 i.e. curve used in bitcoin
#Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424  # y co-ordinate of generating point of secp256k1 i.e. curve used in bitcoin



cnt: 100  neg_cnt: 0


In [ ]:
P = 2 ** 256 - 2 ** 32 - 2 ** 9 - 2 ** 8 - 2 ** 7 - 2 ** 6 - 2 ** 4 - 1
x = (P+1)//4
y= (P+1)/4
print(x)
print(y)

28948022309329048855892746252171976963317496166410141009864396001977208667916
2.894802230932905e+76


In [ ]:
#test y from x
cr = crypto()
pk = cr.generate_private_key()
pubk = cr.private_to_public(pk)
pubk2 = cr.private_to_public(cr.order-pk)
print(pubk)
print(pubk2)
print(pubk[0])
generated_y = cr.ECxToy(pubk[0])
print(generated_y)

#generated_y = cr.ECxToyNotWorking(pubk[0])
#print(generated_y)


(56319530231976224183438377473195172496306287508066867137288889090984035455602, 88592662805159883298140369776025028159328430640934484661820602453851464617457)
(56319530231976224183438377473195172496306287508066867137288889090984035455602, 27199426432156312125430615232662879693941554024706079377636981554057370054206)
56319530231976224183438377473195172496306287508066867137288889090984035455602
(88592662805159883298140369776025028159328430640934484661820602453851464617457, 27199426432156312125430615232662879693941554024706079377636981554057370054206)


In [ ]:
cr = crypto()
cr.P


115792089237316195423570985008687907853269984665640564039457584007908834671663

In [ ]:
cr.Gy

In [ ]:
pk = 0x6385BD34097903323E69A55B32E383C2EB13FFA984222A61D84CDE8DE4703962
pubx,puby = cr.private_to_public(pk)
print(hex(pubx))
print(hex(puby))



0x2a7e7ea4bac6e3c384be6040a030674c78685e2c31fb6f32aab26ebd49ebb951
0xeef5384db9bfee5faf530fca4d2ab8e5b426a6629fcdbddda1c91e830e6197b6


In [ ]:
msg = 0x00C1D0B270F6AC2D9CDC4FC1105D6F1D993D1F2E845FB6F89C705EC6DEB2B3
rn = 0xE7A48DC49D7FE9059E47CBE390E1163EEA5796B4A6AAC77736E9ECFA8500AE48
sign = cr.sign_message(msg,rn,pk)
print(sign)

signing....
rx: 99115877571138904525161819566697840382180511416650522780856252060860694793420
sign: 60068741763705915393223465090797128619537592644105160517755440237969953417130
(99115877571138904525161819566697840382180511416650522780856252060860694793420, 60068741763705915393223465090797128619537592644105160517755440237969953417130)


In [ ]:
print(cr.verify_message(sign[1],sign[0],msg,pubx,puby))

verifying...
m_s: 7654833023648714401928392016443769895283393743334358284275130549657300713275
r_s: 34726424671263813174707202711466381231902374951939214054980053334720705716854
(x,y): (92685075176497724014940311779812843069141506006760926978332164669386080918818, 43942772451509575587469516647184302101407859412812430308898174899807860982857)
(x2,y2): (95412658106290389209996840386726041469899777348374304569468322805084038779545, 17475077123532806769864238591713645415292695891791488591048210907938569122254)
(x3,y3): (99115877571138904525161819566697840382180511416650522780856252060860694793420, 76897929825134835200573813543687982047400288155864558694013583111222351156740)
True


In [ ]:
int(0x4CB32FFDA3EFA569CE7CFD3F9FCFC8D0849AC3015EAA87D430A29D9CA7F0D6)

135517083348175275410893551749236660304032912305017035934790592838100316374